# Обработка данных для функциональности "Определение упоминания организаций (брендов)"

In [ ]:
!pip install corus, nerus
!wget https://github.com/dialogue-evaluation/factRuEval-2016/archive/master.zip
!wget http://www.labinform.ru/pub/named_entities/collection5.zip
!unzip master.zip -d /content/
!unzip collection5.zip -d /content/

In [ ]:
from corus import load_factru, load_ne5

In [ ]:
FACTRU = 'master'
NE5 = 'collection5'
NERUS = 'nerus_lenta.conllu.gz'

In [ ]:
records_factru = load_factru(FACTRU)
next(records_factru)

In [ ]:
records_ne5 = load_ne5(NE5)
next(records_ne5)

In [ ]:
from nerus import load_nerus

docs = load_nerus(NERUS)
doc = next(docs)
doc